In [2]:
deps = [
    "pandas",
    "numpy",
    "torch",
    "scikit-learn",
    "joblib"
]

import pkg_resources
with open("/content/drive/MyDrive/Datathon 2025 (HidupJ0kow1)/requirements_3.txt", "w") as f:
    for pkg in deps:
        try:
            v = pkg_resources.get_distribution(pkg).version
            f.write(f"{pkg}=={v}\n")
        except:
            pass

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    f1_score,
    recall_score,
    roc_auc_score,
    average_precision_score,
    confusion_matrix,
    classification_report,
    precision_recall_curve
    )
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import joblib

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/datathon_test/normal_df.csv')

In [ ]:
label_1 = pd.read_csv('/content/drive/MyDrive/datathon_test/label1_df.csv')

In [ ]:
label_1['label'].value_counts()

,count
label,
1,97644


In [ ]:
data.columns.equals(label_1.columns)

True

In [ ]:
print(label_1.isna().sum())

nan_percent = label_1.isna().mean() * 100
print(nan_percent.sort_values(ascending=False))

sport                       0
dsport                      0
dur                         0
sbytes                      0
dbytes                      0
sttl                        0
dttl                        0
sload                       0
dload                       0
spkts                       0
swin                        0
stcpb                       0
dtcpb                       0
smeansz                     0
dmeansz                     0
trans_depth                 0
res_bdy_len                 0
sjit                        0
djit                        0
sintpkt                     0
dintpkt                     0
tcprtt                      0
is_sm_ips_ports             0
ct_state_ttl                0
ct_flw_http_mthd            0
is_ftp_login                0
ct_srv_src                  0
ct_srv_dst                  0
ct_dst_ltm                  0
ct_src__ltm                 0
ct_src_dport_ltm            0
ct_dst_sport_ltm            0
ct_dst_src_ltm              0
label     

In [ ]:
data_test = pd.concat([data, label_1], ignore_index=True)

In [ ]:
data_test.head()

,sport,dsport,dur,sbytes,dbytes,sttl,dttl,sload,dload,spkts,...,mean_pkt_size,tcp_seq_diff,service_src_ratio,service_dst_ratio,res_bdy_rate,stime_hour,stime_weekday,ltime_hour,ltime_weekday,attack_cat_enc
0,1.210587,0.332228,1.586880,-0.316530,-1.421870,0.0,0.0,-1.405319,-1.985025,-0.736431,...,-0.724248,0.208758,-2.878439,-2.892270,0.0,-1.745615,-1.016005,-1.745633,-1.016004,5
1,0.371878,0.589437,1.598421,0.076287,-0.253445,0.0,0.0,-1.259365,-1.176795,0.006883,...,-0.862612,0.208758,2.090066,0.638164,0.0,-0.311296,-1.016005,-0.311327,-1.016004,5
2,0.590640,1.193551,1.593711,-0.768990,-0.977805,0.0,0.0,-1.598506,-1.681978,-1.004389,...,-0.704652,0.258864,1.244909,0.077801,0.0,0.269936,0.914537,0.269909,0.914530,5
3,1.364205,-1.031797,NaN,-1.479639,-1.175441,0.0,0.0,0.276020,0.060838,-1.397258,...,-1.057971,0.258864,-0.696664,-0.683162,0.0,-0.624466,-1.016005,-0.624497,-1.016004,5
4,-0.715049,1.185669,-0.670657,0.591561,1.261934,0.0,0.0,1.393245,0.078884,0.800095,...,0.757709,0.208758,0.338067,0.374262,0.0,0.937962,0.914537,0.937944,0.914530,5


In [ ]:
data_test['label'].value_counts()

,count
label,
0,1959668
1,97644


In [ ]:
data_test = data_test.fillna(data_test.median())

In [ ]:
data_sampled, _ = train_test_split(data_test, test_size=0.95, stratify=data_test['label'], random_state=42)

In [ ]:
data.head()

,sport,dsport,dur,sbytes,dbytes,sttl,dttl,sload,dload,spkts,...,mean_pkt_size,tcp_seq_diff,service_src_ratio,service_dst_ratio,res_bdy_rate,stime_hour,stime_weekday,ltime_hour,ltime_weekday,attack_cat_enc
0,1.210587,0.332228,1.586880,-0.316530,-1.421870,0.0,0.0,-1.405319,-1.985025,-0.736431,...,-0.724248,0.208758,-2.878439,-2.892270,0.0,-1.745615,-1.016005,-1.745633,-1.016004,5
1,0.371878,0.589437,1.598421,0.076287,-0.253445,0.0,0.0,-1.259365,-1.176795,0.006883,...,-0.862612,0.208758,2.090066,0.638164,0.0,-0.311296,-1.016005,-0.311327,-1.016004,5
2,0.590640,1.193551,1.593711,-0.768990,-0.977805,0.0,0.0,-1.598506,-1.681978,-1.004389,...,-0.704652,0.258864,1.244909,0.077801,0.0,0.269936,0.914537,0.269909,0.914530,5
3,1.364205,-1.031797,NaN,-1.479639,-1.175441,0.0,0.0,0.276020,0.060838,-1.397258,...,-1.057971,0.258864,-0.696664,-0.683162,0.0,-0.624466,-1.016005,-0.624497,-1.016004,5
4,-0.715049,1.185669,-0.670657,0.591561,1.261934,0.0,0.0,1.393245,0.078884,0.800095,...,0.757709,0.208758,0.338067,0.374262,0.0,0.937962,0.914537,0.937944,0.914530,5


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1959668 entries, 0 to 1959667
Data columns (total 58 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   sport                     float64
 1   dsport                    float64
 2   dur                       float64
 3   sbytes                    float64
 4   dbytes                    float64
 5   sttl                      float64
 6   dttl                      float64
 7   sload                     float64
 8   dload                     float64
 9   spkts                     float64
 10  swin                      float64
 11  stcpb                     float64
 12  dtcpb                     float64
 13  smeansz                   float64
 14  dmeansz                   float64
 15  trans_depth               float64
 16  res_bdy_len               float64
 17  sjit                      float64
 18  djit                      float64
 19  sintpkt                   float64
 20  dintpkt                 

In [ ]:
print(data.isna().sum())

nan_percent = data.isna().mean() * 100
print(nan_percent.sort_values(ascending=False))

sport                            0
dsport                           0
dur                         392485
sbytes                           0
dbytes                           0
sttl                             0
dttl                             0
sload                            0
dload                            0
spkts                            0
swin                             0
stcpb                            0
dtcpb                            0
smeansz                     222926
dmeansz                          0
trans_depth                      0
res_bdy_len                      0
sjit                             0
djit                             0
sintpkt                          0
dintpkt                          0
tcprtt                           0
is_sm_ips_ports                  0
ct_state_ttl                     0
ct_flw_http_mthd                 0
is_ftp_login                     0
ct_srv_src                       0
ct_srv_dst                       0
ct_dst_ltm          

In [ ]:
print(data_test.isna().sum())

nan_percent = data_test.isna().mean() * 100
print(nan_percent.sort_values(ascending=False))

sport                       0
dsport                      0
dur                         0
sbytes                      0
dbytes                      0
sttl                        0
dttl                        0
sload                       0
dload                       0
spkts                       0
swin                        0
stcpb                       0
dtcpb                       0
smeansz                     0
dmeansz                     0
trans_depth                 0
res_bdy_len                 0
sjit                        0
djit                        0
sintpkt                     0
dintpkt                     0
tcprtt                      0
is_sm_ips_ports             0
ct_state_ttl                0
ct_flw_http_mthd            0
is_ftp_login                0
ct_srv_src                  0
ct_srv_dst                  0
ct_dst_ltm                  0
ct_src__ltm                 0
ct_src_dport_ltm            0
ct_dst_sport_ltm            0
ct_dst_src_ltm              0
label     

In [ ]:
data = data.fillna(data.median())

In [ ]:
data.head()

,sport,dsport,dur,sbytes,dbytes,sttl,dttl,sload,dload,spkts,...,mean_pkt_size,tcp_seq_diff,service_src_ratio,service_dst_ratio,res_bdy_rate,stime_hour,stime_weekday,ltime_hour,ltime_weekday,attack_cat_enc
0,1.210587,0.332228,1.586880,-0.316530,-1.421870,0.0,0.0,-1.405319,-1.985025,-0.736431,...,-0.724248,0.208758,-2.878439,-2.892270,0.0,-1.745615,-1.016005,-1.745633,-1.016004,5
1,0.371878,0.589437,1.598421,0.076287,-0.253445,0.0,0.0,-1.259365,-1.176795,0.006883,...,-0.862612,0.208758,2.090066,0.638164,0.0,-0.311296,-1.016005,-0.311327,-1.016004,5
2,0.590640,1.193551,1.593711,-0.768990,-0.977805,0.0,0.0,-1.598506,-1.681978,-1.004389,...,-0.704652,0.258864,1.244909,0.077801,0.0,0.269936,0.914537,0.269909,0.914530,5
3,1.364205,-1.031797,-0.091065,-1.479639,-1.175441,0.0,0.0,0.276020,0.060838,-1.397258,...,-1.057971,0.258864,-0.696664,-0.683162,0.0,-0.624466,-1.016005,-0.624497,-1.016004,5
4,-0.715049,1.185669,-0.670657,0.591561,1.261934,0.0,0.0,1.393245,0.078884,0.800095,...,0.757709,0.208758,0.338067,0.374262,0.0,0.937962,0.914537,0.937944,0.914530,5


In [ ]:
print(data.isna().sum())

sport                       0
dsport                      0
dur                         0
sbytes                      0
dbytes                      0
sttl                        0
dttl                        0
sload                       0
dload                       0
spkts                       0
swin                        0
stcpb                       0
dtcpb                       0
smeansz                     0
dmeansz                     0
trans_depth                 0
res_bdy_len                 0
sjit                        0
djit                        0
sintpkt                     0
dintpkt                     0
tcprtt                      0
is_sm_ips_ports             0
ct_state_ttl                0
ct_flw_http_mthd            0
is_ftp_login                0
ct_srv_src                  0
ct_srv_dst                  0
ct_dst_ltm                  0
ct_src__ltm                 0
ct_src_dport_ltm            0
ct_dst_sport_ltm            0
ct_dst_src_ltm              0
label     

In [ ]:
data['label'].value_counts()

,count
label,
0,1959668


In [ ]:
data_normal = data[data['label'] == 0].copy()
X_normal = data_normal.drop(columns=['label', 'attack_cat_enc'])

In [ ]:
X_normal.head()

,sport,dsport,dur,sbytes,dbytes,sttl,dttl,sload,dload,spkts,...,pkt_jit_interaction_src,mean_pkt_size,tcp_seq_diff,service_src_ratio,service_dst_ratio,res_bdy_rate,stime_hour,stime_weekday,ltime_hour,ltime_weekday
0,1.210587,0.332228,1.586880,-0.316530,-1.421870,0.0,0.0,-1.405319,-1.985025,-0.736431,...,1.567628,-0.724248,0.208758,-2.878439,-2.892270,0.0,-1.745615,-1.016005,-1.745633,-1.016004
1,0.371878,0.589437,1.598421,0.076287,-0.253445,0.0,0.0,-1.259365,-1.176795,0.006883,...,1.571804,-0.862612,0.208758,2.090066,0.638164,0.0,-0.311296,-1.016005,-0.311327,-1.016004
2,0.590640,1.193551,1.593711,-0.768990,-0.977805,0.0,0.0,-1.598506,-1.681978,-1.004389,...,0.091105,-0.704652,0.258864,1.244909,0.077801,0.0,0.269936,0.914537,0.269909,0.914530
3,1.364205,-1.031797,-0.091065,-1.479639,-1.175441,0.0,0.0,0.276020,0.060838,-1.397258,...,-1.380182,-1.057971,0.258864,-0.696664,-0.683162,0.0,-0.624466,-1.016005,-0.624497,-1.016004
4,-0.715049,1.185669,-0.670657,0.591561,1.261934,0.0,0.0,1.393245,0.078884,0.800095,...,0.248113,0.757709,0.208758,0.338067,0.374262,0.0,0.937962,0.914537,0.937944,0.914530


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_normal)
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)

In [ ]:
train_dataset = TensorDataset(X_tensor)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, input_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
print(torch.isnan(X_tensor).any())  # cek NaN
print(torch.isinf(X_tensor).any())  # cek inf

tensor(False)
tensor(False)


In [ ]:
input_dim = X_tensor.shape[1]
model = Autoencoder(input_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 50

model.train()
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for (batch,) in train_loader:
        optimizer.zero_grad()
        outputs = model(batch)
        loss = criterion(outputs, batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss/len(train_loader):.6f}")

Epoch 1/50 - Loss: 0.069394
Epoch 2/50 - Loss: 0.045155
Epoch 3/50 - Loss: 0.042906
Epoch 4/50 - Loss: 0.040311
Epoch 5/50 - Loss: 0.039010
Epoch 6/50 - Loss: 0.038546
Epoch 7/50 - Loss: 0.038266
Epoch 8/50 - Loss: 0.038090
Epoch 9/50 - Loss: 0.037935
Epoch 10/50 - Loss: 0.037766
Epoch 11/50 - Loss: 0.037586
Epoch 12/50 - Loss: 0.037429
Epoch 13/50 - Loss: 0.037334
Epoch 14/50 - Loss: 0.036905
Epoch 15/50 - Loss: 0.036270
Epoch 16/50 - Loss: 0.036103
Epoch 17/50 - Loss: 0.035950
Epoch 18/50 - Loss: 0.035644
Epoch 19/50 - Loss: 0.033960
Epoch 20/50 - Loss: 0.032229
Epoch 21/50 - Loss: 0.031714
Epoch 22/50 - Loss: 0.031473
Epoch 23/50 - Loss: 0.031338
Epoch 24/50 - Loss: 0.031222
Epoch 25/50 - Loss: 0.031096
Epoch 26/50 - Loss: 0.030911
Epoch 27/50 - Loss: 0.030761
Epoch 28/50 - Loss: 0.030654
Epoch 29/50 - Loss: 0.030592
Epoch 30/50 - Loss: 0.030553
Epoch 31/50 - Loss: 0.030526
Epoch 32/50 - Loss: 0.030497
Epoch 33/50 - Loss: 0.030475
Epoch 34/50 - Loss: 0.030451
Epoch 35/50 - Loss: 0.0

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/datathon_test/model/autoencoder_model.pth')
print("Model berhasil disimpan ke '/content/drive/MyDrive/datathon_test/model/autoencoder_model.pth'")

Model berhasil disimpan ke '/content/drive/MyDrive/datathon_test/model/autoencoder_model.pth'


In [ ]:
import joblib
joblib.dump(scaler, '/content/drive/MyDrive/datathon_test/model/scaler.pkl')
print("Scaler berhasil disimpan ke '/content/drive/MyDrive/datathon_test/model/scaler.pkl'")

Scaler berhasil disimpan ke '/content/drive/MyDrive/datathon_test/model/scaler.pkl'


In [ ]:
input_dim = X_tensor.shape[1]
model = Autoencoder(input_dim)
model.load_state_dict(torch.load('/content/drive/MyDrive/datathon_test/model/autoencoder_model.pth'))

scaler = joblib.load('/content/drive/MyDrive/datathon_test/model/scaler.pkl')

In [ ]:
X_test = data_sampled.drop(columns=['label', 'attack_cat_enc'])
y_test = data_sampled['label'].values

In [ ]:
print("NaN awal di X_test:", X_test.isnull().sum().sum())

NaN awal di X_test: 0


In [ ]:
X_test_scaled = scaler.transform(X_test)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

In [ ]:
model.eval()

with torch.no_grad():
    X_reconstructed = model(X_test_tensor)
    errors = torch.mean((X_test_tensor - X_reconstructed) ** 2, dim=1).numpy()

In [ ]:
errors_normal = errors[y_test == 0]
threshold = np.percentile(errors_normal, 95)

y_pred = (errors > threshold).astype(int)

In [ ]:
print(f"NaN di X_test_tensor: {torch.isnan(X_test_tensor).sum()}")
print(f"NaN di X_reconstructed: {torch.isnan(X_reconstructed).sum()}")

NaN di X_test_tensor: 0
NaN di X_reconstructed: 0


In [ ]:
f1 = f1_score(y_test, y_pred, pos_label=1)
recall = recall_score(y_test, y_pred, pos_label=1)
roc_auc = roc_auc_score(y_test, errors)
pr_auc = average_precision_score(y_test, errors)

print("==== Evaluation Metrics (Autoencoder Anomaly Detection) ====")
print(f"Recall  (class 1 / anomaly) : {recall:.4f}")
print(f"ROC AUC score               : {roc_auc:.4f}")
print(f"PR AUC (Precision-Recall)   : {pr_auc:.4f}")
print()
print("==== Confusion Matrix ====")
print(confusion_matrix(y_test, y_pred))

==== Evaluation Metrics (Autoencoder Anomaly Detection) ====
Recall  (class 1 / anomaly) : 1.0000
ROC AUC score               : 0.9979
PR AUC (Precision-Recall)   : 0.9534

==== Confusion Matrix ====
[[93083  4900]
 [    0  4882]]
